In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import v2


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as Datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torchvision.utils as vutils
from torch.hub import load_state_dict_from_url
from torchvision import datasets, transforms
from torch.utils.data import random_split

import os
import random
import numpy as np
import math
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import trange, tqdm
from torchvision.transforms import functional as TF
import PIL

latent_channels = 64
batch_size = 64
image_size = 192
lr = 5e-4
nepoch = 100
start_epoch = 0
dataset_root = ""
model_name = "oxford_pets_VAE_Perceptual_03_white_black_silu"
torch.manual_seed(7)  # For reproducibility

save_dir = os.getcwd()
load_checkpoint  = True
im_size = 192

use_cuda = torch.cuda.is_available()
gpu_indx  = 0
device = torch.device(gpu_indx if use_cuda else "cpu")



In [ ]:
import torch
from PIL import Image
from torch.utils.data import Dataset
import os

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, file_names, transform=None, num_black_images=350, num_white_images=350, image_size=(3, 192, 192)):
        self.data_dir = data_dir
        self.transform = transform
        self.file_names = file_names
        self.image_size = image_size

        # Add placeholders for black images
        self.black_image_placeholder = "<black_image>"
        self.file_names.extend([self.black_image_placeholder] * num_black_images)
        self.white_image_placeholder = "<white_image>"
        self.file_names.extend([self.white_image_placeholder] * num_white_images)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        if self.file_names[idx] == self.black_image_placeholder:
            # Create a black image
            black_image = torch.zeros(self.image_size)
            black_image = TF.to_pil_image(black_image)
            return transforms.ToTensor()(black_image) #self.transform(black_image) if self.transform else black_image

        if self.file_names[idx] == self.white_image_placeholder:
            # Create a black image
            white_image = torch.ones(self.image_size)
            white_image = TF.to_pil_image(white_image)
            return transforms.ToTensor()(white_image) #self.transform(white_image) if self.transform else black_image


        img_name = os.path.join(self.data_dir, self.file_names[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

train_transform = transforms.Compose([
    transforms.Resize(size=(im_size, im_size)),
    v2.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([transforms.Resize(size=(im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
#!pip install scikit-learn

from sklearn.model_selection import train_test_split

all_images = [img for img in os.listdir(dataset_root) if img.endswith('.jpg')]
train_images, test_images = train_test_split(all_images, test_size=0.05, random_state=7)

train_dataset = CustomImageDataset(data_dir=dataset_root, file_names=train_images, transform=train_transform, num_black_images=500)
test_dataset = CustomImageDataset(data_dir=dataset_root, file_names=test_images, transform=test_transform, num_black_images=500)




In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(train_loader)
train_images = next(dataiter)
train_images.shape

test_dataiter = iter(test_loader)
test_images = next(test_dataiter)
test_images.shape

In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(test_images[0:64], normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
import torch.nn as nn
import torch


class VGG19(nn.Module):
    """
     Simplified version of the VGG19 "feature" block
     This module's only job is to return the "feature loss" for the inputs
    """

    def __init__(self, channel_in=3, width=64):
        super(VGG19, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, width, 3, 1, 1)
        self.conv2 = nn.Conv2d(width, width, 3, 1, 1)

        self.conv3 = nn.Conv2d(width, 2 * width, 3, 1, 1)
        self.conv4 = nn.Conv2d(2 * width, 2 * width, 3, 1, 1)

        self.conv5 = nn.Conv2d(2 * width, 4 * width, 3, 1, 1)
        self.conv6 = nn.Conv2d(4 * width, 4 * width, 3, 1, 1)
        self.conv7 = nn.Conv2d(4 * width, 4 * width, 3, 1, 1)
        self.conv8 = nn.Conv2d(4 * width, 4 * width, 3, 1, 1)

        self.conv9 = nn.Conv2d(4 * width, 8 * width, 3, 1, 1)
        self.conv10 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)
        self.conv11 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)
        self.conv12 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)

        self.conv13 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)
        self.conv14 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)
        self.conv15 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)
        self.conv16 = nn.Conv2d(8 * width, 8 * width, 3, 1, 1)

        self.mp = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        self.load_params_()

    def load_params_(self):
        # Download and load Pytorch's pre-trained weights
        state_dict = torch.hub.load_state_dict_from_url('https://download.pytorch.org/models/vgg19-dcbb9e9d.pth')
        for ((name, source_param), target_param) in zip(state_dict.items(), self.parameters()):
            target_param.data = source_param.data
            target_param.requires_grad = False

    def feature_loss(self, x):
        return (x[:x.shape[0] // 2] - x[x.shape[0] // 2:]).pow(2).mean()

    def forward(self, x):
        """
        :param x: Expects x to be the target and source to concatenated on dimension 0
        :return: Feature loss
        """
        x = self.conv1(x)
        loss = self.feature_loss(x)
        x = self.conv2(self.relu(x))
        loss += self.feature_loss(x)
        x = self.mp(self.relu(x))  # 64x64

        x = self.conv3(x)
        loss += self.feature_loss(x)
        x = self.conv4(self.relu(x))
        loss += self.feature_loss(x)
        x = self.mp(self.relu(x))  # 32x32

        x = self.conv5(x)
        loss += self.feature_loss(x)
        x = self.conv6(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv7(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv8(self.relu(x))
        loss += self.feature_loss(x)
        x = self.mp(self.relu(x))  # 16x16

        x = self.conv9(x)
        loss += self.feature_loss(x)
        x = self.conv10(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv11(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv12(self.relu(x))
        loss += self.feature_loss(x)
        x = self.mp(self.relu(x))  # 8x8

        x = self.conv13(x)
        loss += self.feature_loss(x)
        x = self.conv14(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv15(self.relu(x))
        loss += self.feature_loss(x)
        x = self.conv16(self.relu(x))
        loss += self.feature_loss(x)

        return loss/16

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data


class ResDown(nn.Module):
    """
    Residual down sampling block for the encoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3):
        super(ResDown, self).__init__()
        self.conv1 = nn.Conv2d(channel_in, channel_out // 2, kernel_size, 2, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_out // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_out // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 2, kernel_size // 2)

        self.act_fnc = nn.SiLU() # Change activation here

    def forward(self, x):
        #skip = self.conv3(x) $ Ignore skip connections
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)
        return self.act_fnc(self.bn2(x))#+ skip))


class ResUp(nn.Module):
    """
    Residual up sampling block for the decoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3, scale_factor=2):
        super(ResUp, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, channel_in // 2, kernel_size, 1, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_in // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_in // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 1, kernel_size // 2)

        self.up_nn = nn.Upsample(scale_factor=scale_factor, mode="nearest")

        self.act_fnc = nn.ELU()

    def forward(self, x):
        x = self.up_nn(x)
        #skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        return self.act_fnc(self.bn2(x)) #+ skip))


class ResBlock(nn.Module):
    """
    Residual block
    """

    def __init__(self, channel_in, channel_out, kernel_size=3, act=nn.ELU()):
        super(ResBlock, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, channel_in // 2, kernel_size, 1, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_in // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_in // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        if not channel_in == channel_out:
            self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 1, kernel_size // 2)
        else:
            self.conv3 = nn.Identity()

        self.act_fnc = act

    def forward(self, x):
        #skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        return self.act_fnc(self.bn2(x))#+ skip))


class Encoder(nn.Module):
    """
    Encoder block
    """

    def __init__(self, channels, ch=64, blocks=(1, 2, 4, 8), latent_channels=512):
        super(Encoder, self).__init__()
        self.conv_in = nn.Conv2d(channels, blocks[0] * ch, 3, 1, 1)

        widths_in = list(blocks)
        widths_out = list(blocks[1:]) + [blocks[-1]]

        layer_blocks = []

        for w_in, w_out in zip(widths_in, widths_out):
            layer_blocks.append(ResDown(w_in * ch, w_out * ch))

        layer_blocks.append(ResBlock(blocks[-1] * ch, blocks[-1] * ch, kernel_size=3, act=nn.SiLU()))
        layer_blocks.append(ResBlock(blocks[-1] * ch, blocks[-1] * ch, kernel_size=3, act=nn.SiLU()))

        self.res_blocks = nn.Sequential(*layer_blocks)

        self.conv_mu = nn.Conv2d(blocks[-1] * ch, latent_channels, 1, 1)
        self.conv_log_var = nn.Conv2d(blocks[-1] * ch, latent_channels, 1, 1)
        self.act_fnc = nn.SiLU() # Change activation here

    def sample(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, sample=False):
        x = self.act_fnc(self.conv_in(x))
        x = self.res_blocks(x)

        mu = self.conv_mu(x)
        log_var = self.conv_log_var(x)

        if self.training or sample:
            x = self.sample(mu, log_var)
        else:
            x = mu

        return x, mu, log_var


class Decoder(nn.Module):
    """
    Decoder block
    Built to be a mirror of the encoder block
    """

    def __init__(self, channels, ch=64, blocks=(1, 2, 4, 8), latent_channels=512):
        super(Decoder, self).__init__()
        self.conv_in = nn.Conv2d(latent_channels, ch * blocks[-1], 1, 1)

        widths_out = list(blocks)[::-1]
        widths_in = (list(blocks[1:]) + [blocks[-1]])[::-1]

        layer_blocks = [ResBlock(blocks[-1] * ch, blocks[-1] * ch, kernel_size=3, act=nn.ELU()),
                        ResBlock(blocks[-1] * ch, blocks[-1] * ch, kernel_size=3, act=nn.ELU())]

        for w_in, w_out in zip(widths_in, widths_out):
            layer_blocks.append(ResUp(w_in * ch, w_out * ch))

        self.res_blocks = nn.Sequential(*layer_blocks)

        self.conv_out = nn.Conv2d(blocks[0] * ch, channels, 3, 1, 1)
        self.act_fnc = nn.ELU()

    def forward(self, x):
        x = self.act_fnc(self.conv_in(x))
        x = self.res_blocks(x)
        mu = torch.tanh(self.conv_out(x))
        return mu


class VAE(nn.Module):
    """
    VAE network, uses the above encoder and decoder blocks
    """
    def __init__(self, channel_in=3, ch=64, blocks=(1, 2, 4, 8), latent_channels=512):
        super(VAE, self).__init__()
        """Res VAE Network
        channel_in  = number of channels of the image
        z = the number of channels of the latent representation
        (for a 64x64 image this is the size of the latent vector)
        """

        self.encoder = Encoder(channel_in, ch=ch, blocks=blocks, latent_channels=latent_channels)
        self.decoder = Decoder(channel_in, ch=ch, blocks=blocks, latent_channels=latent_channels)

    def forward(self, x):
        encoding, mu, log_var = self.encoder(x)
        recon_img = self.decoder(encoding)
        return recon_img, mu, log_var

In [ ]:
import torch.nn.functional as F

def kl_loss(mu, logvar):
    return -0.5 * (1 + logvar - mu.pow(2) - logvar.exp()).mean()

In [ ]:
feature_extractor = VGG19().to(device)

In [ ]:
vae_net = VAE(channel_in=3, ch=64, blocks=(1, 2, 4, 8), latent_channels=64).to(device)
# setup optimizer
optimizer = optim.Adam(vae_net.parameters(), lr=lr, betas=(0.5, 0.999))
#Loss function
loss_log = []
from torchsummary import summary
summary(vae_net, (3,im_size,im_size))

In [ ]:
nepoch = 150

for epoch in trange(start_epoch, nepoch, leave=False):
    train_loss = 0
    train_recon_loss = 0
    train_kld_loss = 0
    train_perceptual_loss = 0

    vae_net.train()
    for i, images in enumerate(tqdm(train_loader, leave=False)):
        images = images.to(device)

        recon_img, mu, logvar = vae_net(images)
        #VAE loss
        kl_loss_ = kl_loss(mu, logvar)
        mse_loss = F.mse_loss(recon_img, images)

        #Perception loss
        feat_in = torch.cat((recon_img, images), 0)
        feature_loss = feature_extractor(feat_in)

        loss = kl_loss_ + mse_loss + feature_loss

        train_loss += loss.item()
        train_recon_loss += mse_loss.item()
        train_kld_loss += kl_loss_.item()
        train_perceptual_loss += feature_loss.item()

        loss_log.append(loss.item())
        vae_net.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = train_loss / len(train_loader)  # Calculate average loss for the epoch
    avg_recon_loss = train_recon_loss / len(train_loader)
    avg_kld_loss = train_kld_loss / len(train_loader)
    avg_perceptual_loss = train_perceptual_loss / len(train_loader)

    #In eval mode the model will use mu as the encoding instead of sampling from the distribution
    vae_net.eval()
    with torch.no_grad():
        recon_img, _, _ = vae_net(test_images.to(device))
        img_cat = torch.cat((recon_img.cpu(), test_images), 2)

        vutils.save_image(img_cat,
                          "%s/%s/%s_%d.png" % (save_dir, "Results" , model_name, image_size),
                          normalize=True)

        #Save a checkpoint
        torch.save({
                    'epoch'                         : epoch,
                    'loss_log'                      : loss_log,
                    'model_state_dict'              : vae_net.state_dict(),
                    'optimizer_state_dict'          : optimizer.state_dict()

                     }, save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt")
    print(f'Epoch {epoch}/{nepoch} - Avg Total Loss: {avg_loss} - Avg Recon Loss: {avg_recon_loss}\
    - Avg KLD Loss: {avg_kld_loss} - Avg Percp Loss: {avg_perceptual_loss}')


In [ ]:
# Run this to load the saved model: load_checkpoint should be True

#Create the save directory if it does note exist
if not os.path.isdir(save_dir + "/Models"):
    os.makedirs(save_dir + "/Models")
if not os.path.isdir(save_dir + "/Results"):
    os.makedirs(save_dir + "/Results")

if load_checkpoint:
    checkpoint = torch.load(save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt", map_location = "cpu")
    print("Checkpoint loaded")
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    vae_net.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint["epoch"]
    loss_log = checkpoint["loss_log"]
else:
    #If checkpoint does exist raise an error to prevent accidental overwriting
    if os.path.isfile(save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt"):
        raise ValueError("Warning Checkpoint exists")
    else:
        print("Starting from scratch")


In [ ]:
vae_net.eval()

black_image = torch.zeros((3,im_size,im_size))
black_image = TF.to_pil_image(black_image)
black_image = test_transform(black_image)
black_image = black_image.unsqueeze(0).cuda()
black_z,_,_ = vae_net.encoder(black_image)
bl = vae_net.decoder(black_z)

out = vutils.make_grid(bl, normalize=True)
plt.imshow(out.cpu().detach().numpy().transpose((1, 2, 0)))


In [ ]:
vae_net.eval()

white_image = torch.ones((3,im_size,im_size))
white_image = TF.to_pil_image(white_image)
white_image = test_transform(white_image)
white_image = white_image.unsqueeze(0)
white_z,_,_ = vae_net.encoder(white_image.cuda())
wl = vae_net.decoder(white_z)

out = vutils.make_grid(wl, normalize=True)
plt.imshow(out.cpu().detach().numpy().transpose((1, 2, 0)))


In [ ]:
# Prepare Dataset for supervised training, without black/white images
# If the model saved, you can start running the notebook here, after the imports in the first cell

import os
from PIL import Image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

train_transform = transforms.Compose([
    transforms.Resize(size=(im_size, im_size)),
    v2.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([transforms.Resize(size=(im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


class CustomImageDatasetWithLabels(Dataset):
    def __init__(self, data_dir, file_names, labels, transform=None):
        self.data_dir = data_dir
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.file_names[idx])
        image = Image.open(img_path).convert('RGB')
        label_name = self.labels[idx]
        label_index = label_to_index[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label_index



def load_dataset(data_dir):
    file_names = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]  # Adjust as needed
    labels = ['_'.join(f.split('_')[:-1]) for f in file_names]
    return file_names, labels

# Load dataset
data_dir = dataset_root
file_names, labels = load_dataset(data_dir)

unique_labels = set()  # A set to store all unique labels

# Assuming 'all_labels' is a list of all labels in the dataset
for label in labels:
    unique_labels.add(label)

# Create a mapping from label to index
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}




# Split dataset
train_files, test_files, train_labels, test_labels = train_test_split(file_names, labels, test_size=0.3, random_state=7)

train_dataset = CustomImageDatasetWithLabels(data_dir, file_names=train_files, labels=train_labels, transform=train_transform)
test_dataset = CustomImageDatasetWithLabels(data_dir, file_names=test_files, labels=test_labels, transform=test_transform)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [ ]:
dataiter = iter(train_loader)
train_images, train_labels = next(dataiter)
train_images.shape

test_dataiter = iter(test_loader)
test_images, test_labels = next(test_dataiter)
test_images.shape

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary

class ResClassifier(nn.Module):
    def __init__(self):
        super(ResClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=True)


        for param in self.resnet.parameters():
            param.requires_grad = False

        self.resnet.fc = nn.Identity()

        # Forward a dummy variable through the feature extractor to determine input size
        dummy_input = torch.randn(1, 3, im_size, im_size)
        dummy_features = self.resnet(dummy_input)
        self.feature_size = dummy_features.view(-1).shape[0]

        self.resnet.fc = nn.Sequential(
            nn.Linear(self.feature_size, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(150, 37),
        )

    def forward(self, x):
        x = self.resnet(x)
        return x

res_classifier = ResClassifier().to(device)

print(res_classifier)


summary(res_classifier, (3, im_size, im_size))


In [ ]:
# Train with frozen weights for 7-10 epochs, then fine-tune the whole model for 3-5 epochs, with lr=0.0001
num_epochs = 7
optimizer = optim.Adam(res_classifier.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()


In [ ]:
vae_net.eval()
# Training loop
def test_model():
    res_classifier.eval()
    val_loss = 0
    val_corrects = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            reconstructed_data, _, _ = vae_net(data)

            output = res_classifier(reconstructed_data)
            val_loss = criterion(output, target)  # Sum up batch loss

            _, predictions = torch.max(output.data, 1)

            val_corrects += torch.sum(predictions.squeeze().int() == target.squeeze().int()).item()

            val_loss += val_loss.item() * data.size(0)

    # Print epoch statistics
    epoch_loss = val_loss / len(test_loader.dataset)
    epoch_acc = float(val_corrects) / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch_loss, val_corrects, len(test_loader.dataset),
        100.0 * float(val_corrects) / len(test_loader.dataset)))

    return epoch_loss

# Existing training loop starts here...
for epoch in range(num_epochs):
    training_corrects = 0
    res_classifier.train()
    training_loss = 0
    total_train = 0
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, leave=False)):
        data, target = data.cuda(), target.cuda()
        #target = target.unsqueeze(1).float()
        reconstructed_data, _, _ = vae_net(data) # Reconstructions from the VAE are fed to the classifier

        optimizer.zero_grad()

        output = res_classifier(reconstructed_data)
        loss = criterion(output, target)

        loss.backward()

        optimizer.step()

        training_loss += loss.item() * data.size(0)
        _, predictions = torch.max(output.data, 1)

        training_corrects += torch.sum(predictions.squeeze().int() == target.squeeze().int()).item()


    epoch_loss = training_loss / len(train_loader.dataset)
    epoch_acc = float(training_corrects) / len(train_loader.dataset)

    print('Epoch: {} Average training loss: {:.4f}, Training Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, epoch_loss, training_corrects, len(train_loader.dataset),
        100.0 * float(training_corrects) / len(train_loader.dataset)))
    # Test the model at the end of each epoch
    test_model()


In [ ]:
for param in res_classifier.parameters():
    param.requires_grad = True

In [ ]:

torch.save({
                    'epoch'                         : epoch,
                    'loss_log'                      : loss_log,
                    'model_state_dict'              : res_classifier.state_dict(),
                    'optimizer_state_dict'          : optimizer.state_dict()

                     }, save_dir + "/Models/" + model_name + "_classifier_" + str(image_size) + ".pt")


In [ ]:
if not os.path.isdir(save_dir + "/Models"):
    os.makedirs(save_dir + "/Models")
if not os.path.isdir(save_dir + "/Results"):
    os.makedirs(save_dir + "/Results")

if load_checkpoint:
    checkpoint = torch.load(save_dir + "/Models/" + model_name + "_classifier_" + str(image_size) + ".pt", map_location = "cpu")
    print("Checkpoint loaded")
    res_classifier.load_state_dict(checkpoint['model_state_dict'])
else:
    #If checkpoint does exist raise an error to prevent accidental overwriting
    if os.path.isfile(save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt"):
        raise ValueError("Warning Checkpoint exists")
    else:
        print("Starting from scratch")

In [ ]:
# Geodesic Path Algorithm

import torch

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)

import gc
gc.collect()
torch.cuda.empty_cache()


def compute_etta(model, zi, zi_minus, zi_plus, dt):
    #model.eval()
    g_zi_minus = model.decoder(zi_minus).view(-1)
    g_zi = model.decoder(zi).view(-1)
    g_zi_plus = model.decoder(zi_plus).view(-1)

    finite_diff = (g_zi_plus - 2*g_zi + g_zi_minus) / dt



    # Use the encoder's Jacobian to map the finite difference in X space back to Z space
    Jh_tuple = torch.autograd.functional.jacobian(model.encoder, g_zi.view(1,3,image_size,image_size))
    Jh = Jh_tuple[0].view(512*4*4, -1)  # Reshape to a 2D tensor

    etta_i = - torch.mm(Jh, finite_diff.unsqueeze(-1)).view_as(zi)
    del Jh
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()

    return etta_i

def compute_etta_d(model, zi, zi_minus, zi_plus, dt):
    # Compute the finite difference
    g_zi_minus = model.decoder(zi_minus).view(-1)
    g_zi = model.decoder(zi).view(-1)
    g_zi_plus = model.decoder(zi_plus).view(-1)

    finite_diff = (g_zi_plus - 2 * g_zi + g_zi_minus) / dt
    finite_diff = finite_diff.view(1, 3, image_size, image_size)  # Reshape it to match the encoder's output shape

    scaled_finite_diff = 0.1 * finite_diff

    # Compute Jacobian-vector product
    vjp_outputs = torch.autograd.functional.vjp(model.decoder, zi, finite_diff)
    #print("vjp", len(vjp_outputs))
    # Get the result from the vjp outputs
    Jv = vjp_outputs[1].view_as(zi)
    #print("Jv", Jv.shape)

    # Compute etta_i
    etta_i = -Jv #+ scaled_finite_diff)

    # Free up memory
    del g_zi_minus, g_zi, g_zi_plus, finite_diff, Jv, vjp_outputs
    torch.cuda.empty_cache()

    return etta_i

def compute_etta1(model, zi, zi_minus, zi_plus, dt):

    g_zi_minus = model.decoder(zi_minus).view(-1).detach()
    g_zi = model.decoder(zi).view(-1)  # We will need the gradient information for this tensor for the Jacobian
    g_zi_plus = model.decoder(zi_plus).view(-1).detach()

    finite_diff = (g_zi_plus - 2 * g_zi + g_zi_minus) / dt

    def partial_encoder(input_data):
        return model.encoder(input_data)[1]

    #encoder_output = model.encoder(model.decoder(zi))  # Assuming input_data is your input to the encoder
      # Access the first element of the tuple
    Jh_tuple = torch.autograd.functional.jacobian(partial_encoder, g_zi.view(1, 3, im_size, im_size))
    Jh = Jh_tuple.view(512 * 4 * 4, -1)  # Reshape to a 2D tensor


    # Use the encoder's Jacobian to map the finite difference in X space back to Z space
    #Jh_tuple = torch.autograd.functional.jacobian(model.encoder, g_zi.view(1, 3, im_size, im_size))
    #Jh = Jh_tuple[0].view(512 * 4 * 4, -1)  # Reshape to a 2D tensor

    etta_i = - torch.mm(Jh, finite_diff.unsqueeze(-1)).view_as(zi)

    # Free up memory
    del g_zi_minus, g_zi, g_zi_plus, Jh, Jh_tuple, finite_diff
    torch.cuda.empty_cache()
    gc.collect()

    return etta_i


def compute_etta(model, zi, zi_minus, zi_plus, dt):
    # Compute the finite difference
    g_zi_minus = model.decoder(zi_minus).view(-1)
    g_zi = model.decoder(zi).view(-1)
    g_zi_plus = model.decoder(zi_plus).view(-1)

    finite_diff = (g_zi_plus - 2 * g_zi + g_zi_minus) / dt
    finite_diff = finite_diff.view(1, 3, image_size, image_size)  # Reshape it to match the encoder's output shape

    # Define a wrapper function for the encoder, so it can handle just the required output
    def partial_encoder(input_data):
        return model.encoder(input_data)[1]

    # Compute Jacobian-vector product
    #vjp_outputs = torch.autograd.functional.jvp(wrapper_decoder, zi, finite_diff)
    vjp_outputs = torch.autograd.functional.jvp(partial_encoder, g_zi.view(1, 3, image_size, image_size), finite_diff)
    # vjp_outputs = torch.autograd.functional.jvp(wrapper_func, g_zi.view(1, 3, 64, 64), finite_diff)

    # Get the result from the vjp outputs
    Jv = vjp_outputs[1].view_as(zi)

    # Compute etta_i
    etta_i = -Jv

    # Free up memory
    del g_zi_minus, g_zi, g_zi_plus, finite_diff, Jv, vjp_outputs
    torch.cuda.empty_cache()
    gc.collect()

    return etta_i


def compute_etta_d(model, zi, zi_minus, zi_plus, dt):
    # Compute the finite difference
    g_zi_minus = model.decoder(zi_minus).view(-1)
    g_zi = model.decoder(zi).view(-1)
    g_zi_plus = model.decoder(zi_plus).view(-1)

    finite_diff = (g_zi_plus - 2 * g_zi + g_zi_minus) / dt
    finite_diff = finite_diff.view(1, 3, image_size, image_size)  # Reshape it to match the encoder's output shape

    # Compute Jacobian-vector product
    vjp_outputs = torch.autograd.functional.vjp(model.decoder, zi, finite_diff)
    #print("vjp", len(vjp_outputs))
    # Get the result from the vjp outputs
    Jv = vjp_outputs[1].view_as(zi)

    # Compute etta_i
    etta_i = -Jv

    # Free up memory
    del g_zi_minus, g_zi, g_zi_plus, finite_diff, Jv, vjp_outputs
    torch.cuda.empty_cache()

    return etta_i

In [ ]:
def backtracking_line_search(model, z_collection, i, direction, start_alpha, beta, dt, max_iterations, c=0.001):
    alpha = start_alpha
    current_energy = sum_of_etta_norms(model, z_collection, dt)
    gradient_norm_square = direction.norm().pow(2)


    tmp_z = z_collection[i] - alpha * direction
    new_z_collection = [element.clone() for element in z_collection]
    new_z_collection[i] = tmp_z
    #iterations_count = 0

    while sum_of_etta_norms(model, new_z_collection, dt) > current_energy - c * alpha * gradient_norm_square:
        alpha *= beta
        tmp_z = z_collection[i] - alpha * direction
        new_z_collection[i] = tmp_z
        #iterations_count+=1

    return alpha

def sum_of_etta_norms(model, z_collection, dt):
    norms = []
    for j in range(1, len(z_collection) - 1):
        etta_j = compute_etta_d(model, z_collection[j], z_collection[j-1], z_collection[j+1], dt)
        norms.append(etta_j.norm().pow(2).item())
        #del etta_j
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
    return sum(norms)


def geodesic_path_algorithm(model, z_collection, alpha, T, beta, epsilon, max_iterations):
    model.eval()
    dt = 1.0 / T
    initial_sum_norms = float('inf')
    iterations = 0

    while sum_of_etta_norms(model, z_collection, dt) > epsilon:
        print(f"It{iterations}:sum_etta_norms", sum_of_etta_norms(model, z_collection, dt))

    #while True:
        etta_norms = []
        if sum_of_etta_norms(model, z_collection, dt) > initial_sum_norms:
                initial_sum_norms = sum_of_etta_norms(model, z_collection, dt)
        else:
            pass
            #break
        #while True:
            #if init == 0:
        if iterations == max_iterations:
            break

        iterations +=1

        for i in range(1, T-1):
            etta_i = compute_etta_d(model, z_collection[i], z_collection[i-1], z_collection[i+1], dt)
            # backtracking_line_search is time consuming, instead use small value for alpha and run for sufficient iterations ~ 200 iterations
            #alpha_i = backtracking_line_search(model, z_collection, i, etta_i, alpha, beta, dt, max_iterations)
            alpha_i= alpha
            z_collection[i] -= alpha_i * etta_i
            etta_norms.append(etta_i.norm().pow(2).item())

            del etta_i
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.empty_cache()
        geodesic_path = z_collection
    return z_collection

# Example usage:
vae_net.eval()
image_size = im_size
x,_,_ =vae_net.encoder(train_images[1].view(1,3,128,128).cuda())
zero_z = torch.FloatTensor(1,64,8,8).zero_().cuda()
#black_image = torch.FloatTensor(1,3,128,128).zero_().cuda()
#black_z, _, _= vae_net.encoder(black_image)
#white_image = torch.ones(1,3,128,128).cuda()
#white_z, _, _= vae_net.encoder(white_image)
#z_0,_,_ = vae_net.encoder(train_images[3].view(1,3,128,128).cuda())



In [ ]:
import torch

def integrated_gradients(model, baseline, image, target_class, steps=20):
    """
    Compute the integrated gradients for a given model and image.

    Parameters:
    - model: the PyTorch model (should be in eval mode)
    - baseline: the baseline image (usually a tensor of zeros with the same shape as the image)
    - image: the input image tensor
    - target_class: the index of the target class for which gradients should be calculated
    - steps: the number of steps in the Riemann sum approximation of the integral

    Returns:
    - Integrated gradients with respect to the input image
    """

    # Ensure model is in evaluation mode
    model.eval()

    # Generate scaled versions of the image
    scaled_images = [baseline + (float(i) / steps) * (image - baseline) for i in range(0, steps)]

    # Convert to tensor
    scaled_images = torch.cat([img.unsqueeze(0) for img in scaled_images], dim=0)

    scaled_images = scaled_images.view(steps, 3,image_size,image_size)

    # Require gradient
    #scaled_images.requires_grad_(True)
    scaled_images = scaled_images.requires_grad_(True)

    # Get model predictions for the scaled images
    model_outputs = model(scaled_images)
    m = torch.nn.Softmax(dim=1)
    model_outputs = m(model_outputs)

    # Extract the scores of the target class
    scores = model_outputs[:, target_class]
    #print(scores[-1])

    # Compute gradients
    #scores.backward(torch.ones_like(scores))

    gradients = torch.autograd.grad(outputs=scores, inputs=scaled_images,
                                    grad_outputs=torch.ones(scores.size()).to(image.device),
                                    create_graph=True)[0]


    # Average the gradients across all steps
    avg_gradients = torch.mean(gradients, dim=0)

    # Compute the integrated gradients
    integrated_gradients = (image - baseline) * avg_gradients

    return integrated_gradients



In [ ]:
import torch

def integrated_gradients_manifold(model, manifold_imgs, target_class, steps=20):
    """
    Compute the integrated gradients for a given model and image.

    Parameters:
    - model: the PyTorch model (should be in eval mode)
    - baseline: the baseline image (usually a tensor of zeros with the same shape as the image)
    - image: the input image tensor
    - target_class: the index of the target class for which gradients should be calculated
    - steps: the number of steps in the Riemann sum approximation of the integral

    Returns:
    - Integrated gradients with respect to the input image
    """

    # Ensure model is in evaluation mode
    model.eval()

    # Generate scaled versions of the image
    scaled_images = manifold_imgs #[baseline + (float(i) / steps) * (image - baseline) for i in range(0, steps)]

    # Convert to tensor
    scaled_images = torch.cat([img.unsqueeze(0) for img in scaled_images], dim=0)

    scaled_images = scaled_images.view(steps, 3,image_size,image_size)

    # Require gradient
    #scaled_images.requires_grad_(True)
    scaled_images = scaled_images.requires_grad_(True)

    # Get model predictions for the scaled images
    model_outputs = model(scaled_images)
    m = torch.nn.Softmax(dim=1)
    model_outputs = m(model_outputs)

    # Extract the scores of the target class
    scores = model_outputs[:, target_class]
    print(scores[-1])

    # Retrieve gradients; gradients are now a tensor of the same shape as scaled_images
    gradients = torch.autograd.grad(outputs=scores, inputs=scaled_images,
                                    grad_outputs=torch.ones(scores.size()).to(scaled_images[0].device),
                                    create_graph=True)[0]


    # Average the gradients across all steps
    avg_gradients = torch.mean(gradients, dim=0)
    baseline = torch.zeros_like(im).cuda()

    # Compute the integrated gradients
    integrated_gradients = (scaled_images[-1] - scaled_images[0]) * avg_gradients

    return integrated_gradients, avg_gradients

# Calculate integrated gradients


In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(test_images[40:60], normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
#8, 29

#43, 49

res_classifier.eval()
ix = 43
im, label = test_images[ix].unsqueeze(0).cuda(), test_labels[ix]
rec_im,_,_ = vae_net(im)
#x = im
output = res_classifier(rec_im)
_, predictions = torch.max(output.data, 1)
target_class = predictions
target_class == label


In [ ]:
#linear path in the latent space
def interpolate(start, end, steps):
    """Generate interpolated vectors between start and end."""
    interpolation = [start + (float(i)/steps)*(end-start) for i in range(0, steps)]
    return interpolation

im = train_images[1].unsqueeze(0).cuda()
vae_net.eval
x_z,_,_ = vae_net.encoder(im)
rand_z = torch.rand_like(x_z)
zeros_z = torch.zeros_like(x_z)
ones_z = torch.ones_like(x_z)

interpolated_vectors = interpolate(black_z, x_z, 20)

interpolated_rec_images = [vae_net.decoder(vec) for vec in interpolated_vectors]

vv = [v.squeeze() for v in interpolated_rec_images]
plt.figure(figsize = (20,10))
out = vutils.make_grid(vv, normalize=True)
plt.imshow(out.cpu().numpy().transpose((1, 2, 0)))


In [ ]:
# geodesic path in the latent space
T = 20
z0 = black_z
zT = x_z
z_collection = [z0 + float(i) / T * (zT - z0) for i in range(T)] # initial linear path
#z_collection = [z0 if i == 0 else zT if i == T - 1 else z0 + float(i) / T * (zT - z0) + 0.8 * torch.randn_like(z0) for i in range(T)] # linear path with added noise
#M = 0.5 * (z0 + zT) $ For cubic path

#z_collection = [((1 - float(i)/T)**2) * z0 + 2*(1 - float(i)/T)*float(i)/T * M + (float(i)/T)**2 * zT for i in range(T)]
path = geodesic_path_algorithm(vae_net, z_collection, alpha=0.00005, T=20, beta=0.5, epsilon=1000, max_iterations=200)



In [ ]:
interpolated_geodesic_images = [vae_net.decoder(vec) for vec in path]

vv = [v.squeeze() for v in interpolated_geodesic_images]
plt.figure(figsize = (20,10))
out = vutils.make_grid(vv, normalize=True)
plt.imshow(out.cpu().numpy().transpose((1, 2, 0)))

In [ ]:
#linear path in the original image space
interpolated_vectors = interpolate(black_image, im, 20)


vv = [v.squeeze() for v in interpolated_vectors]
plt.figure(figsize = (20,10))
out = vutils.make_grid(vv, normalize=True)
plt.imshow(out.cpu().numpy().transpose((1, 2, 0)))

In [ ]:
import torch
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
os.environ['TORCH_HOME'] = 'models\\'
os.getcwd()


In [ ]:
zero_baseline = torch.zeros_like(rec_im).cuda() # Assuming image is your input tensor
to_pil_image = transforms.ToPILImage()
baseline_pil = to_pil_image(zero_baseline[0].cpu())

baseline_transformed = test_transform(baseline_pil)
baseline_transformed = baseline_transformed.unsqueeze(0).cuda()
# Calculate integrated gradients

int_grads_ = integrated_gradients(res_classifier, zero_baseline, rec_im, target_class, steps=20)
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)
_ = viz.visualize_image_attr(np.transpose(int_grads_.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)


In [ ]:
#IG on linear latent
int_grads_lin, av = integrated_gradients_manifold(res_classifier, interpolated_rec_images, target_class, steps=20)
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)

_ = viz.visualize_image_attr(np.transpose(int_grads_lin.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)


In [ ]:
#IG on geodesic latent

int_grads_geo, av = integrated_gradients_manifold(res_classifier, interpolated_geodesic_images, target_class, steps=20)
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)

_ = viz.visualize_image_attr(np.transpose(int_grads_geo.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)

In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(train_images[0:20], normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:

def normalize(exp):
    exp = torch.sum(torch.abs(exp), dim=1)

    exp = exp / torch.sum(exp)
    return exp

def clamp(x, mean, std):
    """
    Helper method for clamping the adversarial example in order to ensure that it is a valid image
    """
    upper = torch.from_numpy(np.array((1.0 - mean) / std)).to(x.device)
    lower = torch.from_numpy(np.array((0.0 - mean) / std)).to(x.device)

    if x.shape[1] == 3:  # 3-channel image
        for i in [0, 1, 2]:
            x[0][i] = torch.clamp(x[0][i], min=lower[i], max=upper[i])
    else:
        x = torch.clamp(x, min=lower[0], max=upper[0])
    return x

data_mean = np.array([0.485, 0.456, 0.406])
data_std = np.array([0.229, 0.224, 0.225])

In [ ]:
# Targeted Attributional attack, see "Explanations can be manipulated and geometry is to blame"



def get_expl(model, x_rec, baseline):

    output = model(x_rec)
    _, predictions = torch.max(output.data, 1)
    target_class = predictions
    m = torch.nn.Softmax(dim=1)
    model_outputs = m(output)
    scores = model_outputs[:, target_class]
    exp = integrated_gradients(model, baseline, x_rec, target_class, steps=20)
    return exp, scores, target_class




x,_,_ = vae_net(test_images[43].unsqueeze(0).cuda())
x_target,_,_ = vae_net(test_images[49].unsqueeze(0).cuda())
x_adv = x.clone().detach().requires_grad_()



baseline = zero_baseline
# produce expls
org_expl, org_acc, org_idx = get_expl(res_classifier, x, baseline)
#org_expl = org_expl.detach().cpu()
target_expl, _, _ = get_expl(res_classifier, x_target, baseline)
target_expl = target_expl.detach()
#target_expl =  normalize(target_expl)
optimizer_adv = torch.optim.Adam([x_adv], lr=0.07)

for i in range(1000):
        optimizer_adv.zero_grad()

        # calculate loss
        adv_expl, adv_acc, class_idx = get_expl(res_classifier, x_adv, baseline)
        #adv_expl = normalize(adv_expl)
        loss_expl = F.mse_loss(adv_expl, target_expl)
        loss_output = F.mse_loss(adv_acc, org_acc.detach())
        input_loss = F.mse_loss(x.detach(), x_adv.detach())
        total_loss = 1.0e17*loss_expl + 5.0e6 *loss_output + 1e3 *input_loss

        # update adversarial example
        total_loss.backward()
        optimizer_adv.step()
        if class_idx != org_idx:
            print("class index changed")
            break

        # clamp adversarial example
        # Note: x_adv.data returns tensor which shares data with x_adv but requires
        #       no gradient. Since we do not want to differentiate the clamping,
        #       this is what we need
        x_adv.data = clamp(x_adv.data, data_mean, data_std)

        print("It-{}: Tot_Lss: {}, Expl_Lss: {}, Out_Lss: {} , inp_Lss: {}".format(i, total_loss.item(),
                                                                                   loss_expl.item(),
                                                                                   loss_output.item(),input_loss.item()))

    # test with original model (with relu activations)
adv_expl, adv_acc, class_idx = get_expl(res_classifier, x_adv, baseline)


In [ ]:
#Check if adversarial image still has the same output
class_idx == org_idx


In [ ]:
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)

_ = viz.visualize_image_attr(np.transpose(adv_expl.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)

In [ ]:

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)

_ = viz.visualize_image_attr(np.transpose(org_expl.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)

In [ ]:
target_expl, _, _ = get_expl(res_classifier, x_target, baseline)
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#000000'),
                                                  (0.25, '#ffffff'),
                                                  (1, '#ffffff')], N=256)

_ = viz.visualize_image_attr(np.transpose(target_expl.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(im.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='absolute_value',
                             outlier_perc=1)

In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(x_adv.squeeze().cpu().detach(), normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(x.squeeze().cpu().detach(), normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))
